In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import math
from datetime import date
import datetime
from functools import reduce
import warnings
warnings.filterwarnings("ignore")

In [62]:
personal_data = pd.read_csv('./data/personal_train.csv')
personal_data = personal_data.drop(['Unnamed: 0'], axis=1)
other_data = pd.read_csv('./data/other_train.csv')
other_data = other_data.drop(['Unnamed: 0'], axis=1)

merged = pd.merge(personal_data,other_data,on=["name","address"])
duplicated = merged.duplicated(subset=["name","address"],keep=False)
duplicates = pd.DataFrame(columns=merged.columns)
indexes_to_drop = []
merged_without_duplicates = pd.DataFrame(columns=merged.columns)
for index, row in merged.iterrows():
    if(duplicated[index]):
        duplicates = duplicates.append(merged.iloc[index],ignore_index=True)
        indexes_to_drop.append(index)

duplicates.sort_values(["name","address"])



merged_without_duplicates = merged

for index in indexes_to_drop:
    merged_without_duplicates = merged_without_duplicates.drop(merged.index[index])

merged = merged_without_duplicates

def checkNaN(arr):
    for item in arr:
        if((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item)):
            return False
    return True

def checkQuestionMark(arr):
    for item in arr:
        if(item is not '?'):
            return False
    return True

def nanOrQuestionMark(arr):
    for item in arr:
        if(item is not '?' and ((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item))):
            return False
    return True

def filter_nan_question_mark(arr):
    result = []
    for item in arr:
        if(item is not '?' and ((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item))):
            result.append(item)
    return result        

def getUniqueRow(startIndex,endIndex):
    row_dict = {}

    for column in duplicates.columns: 
        temp = []
        for index in range(startIndex,endIndex):
            temp.append(duplicates.iloc[index][column])
         
        if(checkNaN(temp)):
            row_dict[column] = np.nan
        elif(checkQuestionMark(temp)):
            row_dict[column] = '?'
        elif(nanOrQuestionMark(temp)):
            row_dict[column] = '?'
        else:
            temp = filter_nan_question_mark(temp)
            value = randint(0,len(temp)-1)
            row_dict[column] = temp[value]
    return row_dict
    
count = 0
lastDuplicate = duplicates.iloc[0]["name"]
columns = duplicates.columns.values
for index,row in duplicates.iterrows():
    entry = duplicates.iloc[index]
    name = entry["name"]
    if(name == lastDuplicate):
        count += 1
    else:
        merged = merged.append(getUniqueRow(index-count,index),ignore_index=True)
        lastDuplicate = name
        count = 1
        
    if(len(duplicates) - 1 == index):
        merged = merged.append(getUniqueRow(index-count+1,index + 1),ignore_index=True)
        
        
for index, row in merged.iterrows():
    newRow = eval(row["medical_info"])
    for key, value in newRow.items():
        merged.loc[index,key] = value

merged = merged.drop("medical_info",axis=1)

for column in merged.columns:
    merged[column] = merged[column].map(lambda x: np.nan if '?' in str(x) or 'nan' == str(x) else x)

averageAge = np.nanmean(merged["age"].tolist())

newDate = datetime.datetime(math.floor(datetime.datetime.now().year - averageAge), datetime.datetime.now().month, datetime.datetime.now().day,
                           datetime.datetime.now().hour, datetime.datetime.now().minute).date()

def format_date(string):
    for fmt in ["%Y/%m/%d", "%Y%m%d", "%y-%m-%d", "%Y-%m-%d", "%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H %M %S", "%d/%m/%Y"]:
        try:
            return datetime.datetime.strptime(string, fmt).date()
        except ValueError:
            continue
    raise ValueError(string)

# Funkcia vráti všetky unikátne hodnoty stĺpca
def getUniqueValues(arr):
    return arr.value_counts(normalize=True, dropna=True).index

# Funkcia vráti pomery unikátnych hodnôt stĺpca
def getRatio(arr):
    return arr.value_counts(normalize=True, dropna=True).values

# Nahradí NaN za jednu z hodnôt stĺpca tak, aby pomer hodnôt zostal zachovaný
def fillNanAccordingToRatio(arr):
    return arr.fillna(pd.Series(np.random.choice(getUniqueValues(arr), p=getRatio(arr), size=len(arr))))

#naformatuje datum
merged["date_of_birth"] = merged["date_of_birth"].map(lambda x: format_date(x) if type(x) is str else x)
#doplni chybajuce veky s priemerom
merged["age"] = merged["age"].map(lambda x: math.floor(averageAge) if str(x) == 'nan' or x > 109 else x)
#doplni chybajuce datumy na zaklade priemerneho veku
merged["date_of_birth"] = merged["date_of_birth"].map(lambda x: newDate if str(x) == 'nan' else x)
#opravi datumy ktore su vacsie ako aktualne na zaklade veku

#opravi datumy ktore su vacsie ako aktualne na zaklade veku
i = 0
for date in merged["date_of_birth"]:
    if(date > datetime.datetime.now().date()):
        modifiedDate = datetime.datetime(datetime.datetime.now().year-int(merged["age"][i]), date.month, date.day).date()
        merged["date_of_birth"][i] = modifiedDate 
    i += 1
    
    
    #doplni chybajuce TT4 pomocou algoritmu k najblizsich
k = 5
kNearest = []

i = 0
for tt4 in merged["TT4"]:
    if(str(tt4) == 'nan'):
        kNearest = []
        for index in range(2*k+1):
            if(i+k-index < len(merged) and i+k-index > -1 and i+k-index != i and str(merged["TT4"][i+k-index]) != 'nan' and merged["T4U"][i+k-index] != '?'):
                kNearest.append(merged["TT4"][i+k-index])
        nearestAvg = reduce(lambda x, y: x + y, kNearest) / len(kNearest)
        merged["TT4"][i] = math.floor(nearestAvg)
    i += 1
    
    
#doplni chybajuce T4U pomocou algoritmu k najblizsich
k = 5
kNearest = []

i = 0
for tt4 in merged["T4U"]:
    if(str(tt4) == 'nan'):
        kNearest = []
        for index in range(2*k+1):
            if(i+k-index < len(merged) and i+k-index > -1 and i+k-index != i and str(merged["T4U"][i+k-index]) != 'nan' and merged["T4U"][i+k-index] != '?'):
                kNearest.append(merged["T4U"][i+k-index])
        nearestAvg = reduce(lambda x, y: x + y, kNearest) / len(kNearest)
        merged["T4U"][i] = nearestAvg
    i += 1
    
#doplni chybajuce T4U pomocou algoritmu k najblizsich
k = 5
kNearest = []

i = 0
for tt4 in merged["TSH"]:
    if(str(tt4) == 'nan'):
        kNearest = []
        for index in range(2*k+1):
            if(i+k-index < len(merged) and i+k-index > -1 and i+k-index != i and str(merged["TSH"][i+k-index]) != 'nan' and merged["TSH"][i+k-index] != '?'):
                kNearest.append(merged["TSH"][i+k-index])
        nearestAvg = reduce(lambda x, y: x + y, kNearest) / len(kNearest)
        merged["TSH"][i] = nearestAvg
    i += 1

median = np.nanmedian(merged["T3"].tolist())

merged["T3"] = merged["T3"].map(lambda x: median if str(x) == 'nan' or x == '?' else x)

median = np.nanmedian(merged["education-num"].tolist())

merged["education-num"] = merged["education-num"].map(lambda x: median if str(x) == 'nan' else x)
def mapTrueFalse(x):
    if("t" in str(x).lower()):
        return "t"
    elif("f" in str(x).lower()):
        return "f"
    else:
        return np.nan
true_false_columns = ["sick","T3 measured","query hypothyroid", "query hyperthyroid","FTI measured","lithium","TT4 measured", "pregnant",
                     "thyroid surgery","TSH measured","query on thyroxine","I131 treatment","on thyroxine","psych", "T4U measured", "tumor",
                     ]

for column in true_false_columns:
    merged[column] = merged[column].map(lambda x: mapTrueFalse(x))
    
average_columns = ["age","TT4","T4U","capital-loss","TSH","FTI"]
for column in average_columns:
    merged[column] = merged[column].map(lambda x: float(x))
    average = np.nanmean(merged[column].tolist())
    merged[column] = merged[column].map(lambda x: math.floor(average) if 'nan' in str(x).lower() else x)
    
merged["sick"] = fillNanAccordingToRatio(merged["sick"])
    

In [63]:

averageCapital = np.nanmean(merged["capital-loss"].tolist())

merged["sex"] = fillNanAccordingToRatio(merged["sex"])
merged["workclass"] = fillNanAccordingToRatio(merged["workclass"])
merged["native-country"] = fillNanAccordingToRatio(merged["native-country"])
merged["occupation"] = fillNanAccordingToRatio(merged["occupation"])
merged["T3 measured"] = fillNanAccordingToRatio(merged["T3 measured"])
merged = merged.drop("TBG",axis=1)
merged = merged.drop("TBG measured",axis=1)


In [64]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 43 columns):
name                         2237 non-null object
address                      2237 non-null object
age                          2237 non-null float64
sex                          2237 non-null object
date_of_birth                2237 non-null object
query hyperthyroid           2237 non-null object
FTI measured                 2237 non-null object
education                    2237 non-null object
lithium                      2237 non-null object
TT4                          2237 non-null float64
T4U                          2237 non-null float64
capital-loss                 2237 non-null float64
capital-gain                 2237 non-null float64
tumor                        2237 non-null object
TSH                          2237 non-null float64
T3                           2237 non-null float64
fnlwgt                       2237 non-null float64
hours-per-week               2237 n